In [2]:
# ymyD7ijqAGHxBJYOpjrHZtlC2mHZYEN6
# ygqDyviPRT7ZOkLP

from amadeus import Client, ResponseError
import requests
from math import sin, cos, sqrt, atan2, radians
import pandas as pd
import numpy as np
import sqlite3
import json
import traceback
from sqlite3 import IntegrityError
import csv
import datetime
import functools

amadeus = Client(
    client_id='ymyD7ijqAGHxBJYOpjrHZtlC2mHZYEN6',
    client_secret='ygqDyviPRT7ZOkLP'
)


In [3]:
def create_connection(db_file="event_travel.db"):
    conn = None
    try:
        conn = sqlite3.connect(db_file)
    except Error as e:
        print(e)

    return conn

In [4]:
def execute_select_query(q,cur,conn):
    try:
        cur.execute(q)
    except IntegrityError:
        print('Found a duplicate already in the DB.')
#         traceback.print_exc()   

def execute_insert_query(q,val,cur,conn):
    try:
        cur.execute(q, val)
        conn.commit()
    except IntegrityError:
        print('Found a duplicate already in the DB.')
#         traceback.print_exc()    


In [5]:
def get_distance(lat1, lon1, lat2, lon2):
    
    # approximate radius of earth in km
    R = 6373.0
    Miles_per_Km = 0.621371

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c * Miles_per_Km

    return distance

In [6]:
def find_closest_airports(lat, long, r):
    ret = []
    best = r
    
    conn = create_connection()
    cur = conn.cursor()
    execute_select_query(f'select * from airports',cur,conn)
    airports = cur.fetchall()
    conn.close()

    for airport in airports:

        distance = get_distance(lat, long, airport[4], airport[5])
        if distance <= r:
            data = {'id':airport[0],'name':airport[1],'distance':distance,'airport':airport[3]}
            if distance < best:
                ret.insert(0,data)
                best = distance
            else:
                ret.append(data)
                
    
    return sorted(ret, key=lambda k: k['distance'])             

In [7]:
where_are_you = 'SLC'

In [8]:
likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT'],
     [36.085044, -115.149928, 'Las Vegas', 'NV'],
     [39.851704, -104.673811, 'Denver', 'CO']
]
for dept in likely_departure_points:
    print(dept)
    ret = find_closest_airports(dept[0],dept[1],30)
    print(len(ret))
    for x in ret:
        print(x)

[40.790152, -111.979038, 'Salt Lake City', 'UT']
1
{'id': '3536', 'name': 'Salt Lake City International Airport', 'distance': 0.13286185225979918, 'airport': 'SLC'}
[36.085044, -115.149928, 'Las Vegas', 'NV']
1
{'id': '3877', 'name': 'McCarran International Airport', 'distance': 0.360713200823457, 'airport': 'LAS'}
[39.851704, -104.673811, 'Denver', 'CO']
1
{'id': '3751', 'name': 'Denver International Airport', 'distance': 0.6920960602071604, 'airport': 'DEN'}


In [9]:
def extract_event_json(event):
    try:
        name = event['name']
        event_id = event['id']

        venue = event['_embedded']['venues'][0]
        location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

        latlong = location['latlong']
        airports = find_closest_airports(latlong[0], latlong[1], 20)
        event['airports'] = airports
    except KeyError:
        print('KeyError')
#         traceback.print_exc()
        event = {}

    return event


In [10]:
def find_events_by_classification(page_number=0, classification='music'):
    remaining_pages = True
    total_pages = 0
    page_count = 0
    ret = []
    conn = create_connection()
    cur = conn.cursor()
    while remaining_pages:
        try:
            URL = f"https://app.ticketmaster.com/discovery/v2/events.json"
            PARAMS = {'apikey':'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page':page_number, 
                      'countryCode':'US', 'classificationName':classification,'size':25} 
            print(PARAMS)
            r = requests.get(url = URL, params = PARAMS) 
            data = r.json()
            page = data['page']
            size = page['size']
            page_number = page['number']
            total_pages = page['totalPages']
            events = data['_embedded']['events']
            print(f'page: {page_number}/{total_pages}')
            #         print(len(events))
            for event in events:
                event_json = extract_event_json(event)
                if len(event_json) > 0:
                    name = event_json['name']
                    if 'priceRanges' in event_json:
            #             print(name)
                        line = (event_json['id'], name, classification, json.dumps(event_json))
                        execute_insert_query('insert into raw_events(event_id, name, classification, json) values(?,?,?,?)', line,cur,conn)
        except KeyError:
            traceback.print_exc()
        except IntegrityError:
            traceback.print_exc()

        remaining_pages = ( page_number * size ) < 300
        page_number += 1
        
    conn.close()

 

In [27]:
find_events_by_classification(0,'music')

{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 0, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 0/620
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 1, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 1/620
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 2, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 2/620
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 3, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 3/620
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 4, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 4/620
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page': 5, 'countryCode': 'US', 'classificationName': 'music', 'size': 25}
page: 5/620
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
KeyError
{'apikey': 'kNkNPU0S8LOzOUrjDWpLto6LEv91lAFM', 'page

In [28]:
def get_event_names(from_raw=True):
    conn = create_connection()
    cur = conn.cursor()
    if from_raw:
        cur.execute(f'select distinct name from raw_events order by name')
    else:
        cur.execute(f'select distinct name from my_events order by name')
    names = cur.fetchall()
    conn.close()
    for name in names:
        print(name[0])

In [36]:
get_event_names()

 Alanis Morissette w/special guest Garbage & also appearing Liz Phair
2020 B105 Country Megaticket
2020 Country Megaticket
2020 Jiffy Lube Live Country Megaticket
2020 Jones Beach Country Megaticket
2020 KSON Country Megaticket
2020 Pnc Bank Arts Center Country Megaticket Presented By Pennzoil
2021 CMA Fest
A Thousand Thoughts: A live documentary with the Kronos Quartet
Alanis Morissette w/ special Guest Garbage & also appearing Liz Phair
Alanis Morissette w/special guest Garbage & also appearing Liz Phair
Alanis Morissette w/special guest Garbage & also sppearing Liz Phair
An Evening with Michael Buble in Concert
Backstreet Boys: DNA World Tour
Billy Joel
Billy Joel - In Concert
Boo Fest 2 featuring Kevin Gates and Friends
Buckeye Country Superfest w/ Kenny Chesney, Florida Georgia Line & mor
Celine Dion: Courage World Tour
Cher: Here We Go Again Tour
Chris Stapleton's All-American Roadshow
Chris Stapleton: Concert for Kentucky, An Outlaw State of Kind Benefit
Concert for Peace
Countr

In [49]:
def store_event(event, my_location):
    
    conn = create_connection()
    cur = conn.cursor()
    
    dates = event['dates']['start']
    event_id = event['id']
    sales = event['sales']['public']

    prices = event['priceRanges']
    venue = event['_embedded']['venues'][0]
    location = {'address':venue['address']['line1'],'city':venue['city']['name'],'state':venue['state']['stateCode'],
                'latlong':[float(venue['location']['latitude']),float(venue['location']['longitude'])]}

    event_date = dates['localDate']
    event_time = dates['localTime']
    price_range = [prices[0]['min'],prices[0]['max']]
    distance = get_distance(my_location[0],my_location[1],location['latlong'][0],location['latlong'][1])
    latlong = location['latlong']
    airports = find_closest_airports(latlong[0], latlong[1], 20)

    
    airport_count = len(airports)
    within_year = datetime.datetime.strptime(event_date,"%Y-%m-%d") < datetime.datetime.today() + datetime.timedelta(days=350)
    print(f"Airports: {airport_count} date: {event_date} within_year: {within_year}")
    
    if airport_count > 0 and \
    datetime.datetime.strptime(event_date,"%Y-%m-%d") > datetime.datetime.today() and \
    datetime.datetime.strptime(event_date,"%Y-%m-%d") < datetime.datetime.today() + datetime.timedelta(days=350):
    
        val = (event_id,
               event['name'],
               event_date,
               event_time,
               price_range[0],
               price_range[1],
               location['address'],
               location['city'],
               location['state'],
               latlong[0],
               latlong[1],
               distance,
               my_location[0],
               my_location[1],
               my_location[2],
               my_location[3],
               my_location[4],
               airports[0]['airport']
              )
        
        print(f"storing {val} at {my_location}")
        execute_insert_query("""INSERT INTO my_events(
        id,name,date,time,price_low,price_high,street_address,
        city,state,lat,long,distance,my_lat,my_long,my_city,my_state,depart_airport,dest_airport) 
        VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?)""",val,cur,conn)

#     for airport in airports:
#         val2 = (event_id ,airport['airport'],airport['distance'])
#         execute_insert_query('insert into airports(event_id, airport, distance) values(?,?,?)',val2,cur,conn)

    conn.close()


In [50]:
def select_events(keyword, my_location):
    conn = create_connection()
    cur = conn.cursor()
    cur.execute(f'select * from raw_events where name like "%{keyword}%"')
    events = cur.fetchall()
    conn.close()
    
    for event in events:
        name = event[1]
        print(name)
        event_json = json.loads(event[3])
        store_event(event_json, my_location)    

In [53]:
preferred_event_names = ['Shania Twain', 'Billy Joel']

likely_departure_points = [
    [40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC']]
#     [36.085044, -115.149928, 'Las Vegas', 'NV'],
#     [39.851704, -104.673811, 'Denver', 'CO']]

for event_name in preferred_event_names:
    for location in likely_departure_points:
        print(f'Getting {event_name} from {location[2]}')
        select_events(event_name,location)

Getting Shania Twain from Salt Lake City
Shania Twain "Let's Go!" - The Las Vegas Residency
Airports: 1 date: 2020-08-21 within_year: True
storing ('1AyZA-7GkdJTD8H', 'Shania Twain "Let\'s Go!" - The Las Vegas Residency', '2020-08-21', '20:00:00', 60.0, 251.0, '3667 S Las Vegas Blvd', 'Las Vegas', 'NV', 36.10961689, -115.17247058, 366.70987218488904, 40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC', 'LAS') at [40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC']
Found a duplicate already in the DB.
Shania Twain "Let's Go!" - The Las Vegas Residency
Airports: 1 date: 2020-12-02 within_year: True
storing ('1AyZA-7GkdEk5oG', 'Shania Twain "Let\'s Go!" - The Las Vegas Residency', '2020-12-02', '20:00:00', 60.0, 251.0, '3667 S Las Vegas Blvd', 'Las Vegas', 'NV', 36.10961689, -115.17247058, 366.70987218488904, 40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC', 'LAS') at [40.790152, -111.979038, 'Salt Lake City', 'UT', 'SLC']
Found a duplicate already in the DB.
Shania Twain "Let's

In [54]:
def get_my_events(order_by):
    conn = create_connection()
    cur = conn.cursor()
    cur.execute(f'select name, price_low, price_high, distance, date, city, state, depart_airport, dest_airport from my_events {order_by}')
    out = cur.fetchall()
    conn.close()
    return out



In [18]:
def get_flight_search_json(origin, destination, event_date):

    d = datetime.datetime.strptime(event_date,"%Y-%m-%d")
    delta = datetime.timedelta(days=1)
    start = datetime.datetime.strftime(d - delta, "%Y-%m-%d")
    end = datetime.datetime.strftime(d + delta, "%Y-%m-%d")
    
    return {
              "currencyCode": "USD",
              "originDestinations": [
                {
                  "id": "1",
                  "originLocationCode": origin,
                  "destinationLocationCode": destination,
                  "departureDateTimeRange": {
                    "date": start,
                    "time": "8:00:00"
                  }
                },
                {
                  "id": "2",
                  "originLocationCode": destination,
                  "destinationLocationCode": origin,
                  "departureDateTimeRange": {
                    "date": end,
                    "time": "17:00:00"
                  }
                }
              ],
              "travelers": [
                {
                  "id": "1",
                  "travelerType": "ADULT"
                }
              ],
              "sources": [
                "GDS"
              ],
              "searchCriteria": {
                "maxFlightOffers": 1

              }
            }

In [55]:
@functools.lru_cache(maxsize=128)
def get_flights(origin, destination, event_date):
    print(f"Getting flights from {origin} to {destination} on {event_date}")

    body = get_flight_search_json(origin, destination, event_date)
    response = amadeus.shopping.flight_offers_search.post(body)
    return response.data

In [61]:
flight_results = get_flights('SLC', 'SFO', '2021-05-01')

Getting flights from SLC to SFO on 2021-05-01


In [60]:
flight_results

[{'type': 'flight-offer',
  'id': '1',
  'source': 'GDS',
  'instantTicketingRequired': False,
  'nonHomogeneous': False,
  'oneWay': False,
  'lastTicketingDate': '2020-06-19',
  'numberOfBookableSeats': 7,
  'itineraries': [{'duration': 'PT1H29M',
    'segments': [{'departure': {'iataCode': 'SLC',
       'terminal': '2',
       'at': '2021-04-30T15:05:00'},
      'arrival': {'iataCode': 'DEN', 'at': '2021-04-30T16:34:00'},
      'carrierCode': 'DL',
      'number': '1277',
      'aircraft': {'code': '738'},
      'operating': {'carrierCode': 'DL'},
      'duration': 'PT1H29M',
      'id': '1',
      'numberOfStops': 0,
      'blacklistedInEU': False}]},
   {'duration': 'PT1H37M',
    'segments': [{'departure': {'iataCode': 'DEN',
       'at': '2021-05-02T17:15:00'},
      'arrival': {'iataCode': 'SLC',
       'terminal': '2',
       'at': '2021-05-02T18:52:00'},
      'carrierCode': 'DL',
      'number': '1277',
      'aircraft': {'code': '738'},
      'operating': {'carrierCode': 'D

In [22]:
t_flight = flight_results[0]['itineraries'][0]['segments'][0]['departure']
r_flight = flight_results[0]['itineraries'][1]['segments'][0]['departure']
print(f"depart:{t_flight['iataCode']} at:{t_flight['at']} ret:{r_flight['iataCode']} at:{r_flight['at']} ${flight_results[0]['price']['grandTotal']}")

depart:SLC at:2021-04-30T15:05:00 ret:DEN at:2021-05-02T17:15:00 $204.20


In [56]:
my_events = get_my_events("order by distance, price_low, date")
with open('my_events.csv', mode='w') as events_file:
    event_writer = csv.writer(events_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    current_event = ''
    for event in my_events:
        response = get_flights(event[7], event[8], event[4])
        if len(response) == 0:
            print(f"Can't find this flight: {event[7]} {event[8]} {event[4]}")
        else:
            flight_results = response[0]
            t_flight = flight_results['itineraries'][0]['segments'][0]['departure']
            r_flight = flight_results['itineraries'][1]['segments'][0]['departure']
            low = event[1] + float(flight_results['price']['grandTotal'])
            high = event[2] + float(flight_results['price']['grandTotal'])
            if current_event != event[0]:
                print(event[0])
                current_event=event[0]
            print(f"total cost:${low:.2f} to ${high:.2f} event tickets:${event[1]:.2f} to ${event[2]:.2f} flight:${flight_results['price']['grandTotal']} distance:{event[3]:.2f} date:{event[4]}, city:{event[5]}, state:{event[6]}, depart:{t_flight['iataCode']} at:{t_flight['at']} ret:{r_flight['iataCode']} at:{r_flight['at']}")
            event_writer.writerow(event)
        

Getting flights from SLC to LAS on 2020-08-21
Shania Twain "Let's Go!" - The Las Vegas Residency
total cost:$176.20 to $367.20 event tickets:$60.00 to $251.00 flight:$116.20 distance:366.71 date:2020-08-21, city:Las Vegas, state:NV, depart:SLC at:2020-08-20T20:58:00 ret:LAS at:2020-08-22T17:25:00
Getting flights from SLC to LAS on 2020-08-22
total cost:$307.20 to $498.20 event tickets:$60.00 to $251.00 flight:$247.20 distance:366.71 date:2020-08-22, city:Las Vegas, state:NV, depart:SLC at:2020-08-21T20:58:00 ret:LAS at:2020-08-23T19:00:00
Getting flights from SLC to LAS on 2020-08-26
total cost:$196.20 to $387.20 event tickets:$60.00 to $251.00 flight:$136.20 distance:366.71 date:2020-08-26, city:Las Vegas, state:NV, depart:SLC at:2020-08-25T17:30:00 ret:LAS at:2020-08-27T19:00:00
Getting flights from SLC to LAS on 2020-08-28
total cost:$176.20 to $367.20 event tickets:$60.00 to $251.00 flight:$116.20 distance:366.71 date:2020-08-28, city:Las Vegas, state:NV, depart:SLC at:2020-08-27T2

In [33]:
get_flights.cache_info()

CacheInfo(hits=14, misses=18, maxsize=128, currsize=15)